# Quickstart

## Installation

```bash
pip install fastrepl
```

You can find all releases [here](https://pypi.org/project/fastrepl).

## Setup FastREPL

In [1]:
# When using it in a script
import fastrepl

# When using it in a notebook
import fastrepl.repl as fastrepl

## Prepare Dataset

We will use [daily_dialog](https://huggingface.co/datasets/daily_dialog) from Huggingface.

In [2]:
import re
from datasets import load_dataset

ds = load_dataset("daily_dialog", split="test")
ds = ds.shuffle(4)
ds = ds.select(range(120))


def clean(text):
    return re.sub(r"\s+([,.'!?])", r"\1", text.strip())


def get_input(row):
    msgs = [clean(msg) for msg in row["dialog"]]
    row["input"] = "\n".join(msgs)
    return row


ds = ds.map(get_input, remove_columns=["dialog", "act", "emotion"])
ds

Dataset({
    features: ['input'],
    num_rows: 120
})

## Define Evaluator

Here, we are doing simple classifiction, but there are two interesting points.

1. You can pass nearly any model for evaluation. (Thanks to [LiteLLM](https://github.com/BerriAI/litellm)).
2. **`fastrepl` enhances accuracy by reducing [bias](/guides/dealing_with_bias.md)**. `position_debias_strategy` is one example, which ensures that the order of labels doesn't affect the outcome.

In [3]:
evaluator = fastrepl.Evaluator(
    pipeline=[
        fastrepl.LLMClassificationHead(
            model="gpt-3.5-turbo",
            context="You will receive casual conversation between two people.",
            labels={
                "FUN": "at least one of the two people try to be funny and entertain.",
                "NOT_FUN": "given conversation lacks humor or entertainment value.",
            },
            position_debias_strategy="consensus",
        ),
    ]
)

## Run Evaluator

Here are some notes about running the evaluator:

1. `ThreadPool` is used to make it faster (controlled by the `NUM_THREADS` [environment variable](/getting_started/env.md)).
2. Any errors from different LLM providers are properly handled and retried with backoff if necessary.
3. Since we passed `num=2` to `run()`, it will execute same evaluation twice, and return two results. If there are high inconsistency between the two, you'll see [InconsistentPredictionWarning](/miscellaneous/warnings_and_errors).

In [4]:
result = fastrepl.LocalRunner(evaluator=evaluator, dataset=ds).run(num=2)
result

Output()

/Users/yujonglee/dev/fastrepl/fastrepl/fastrepl/warnings.py:24: CompletionTruncatedWarning: B | 
https://docs.fastrepl.com/miscellaneous/warnings_and_errors#completiontruncated

/Users/yujonglee/dev/fastrepl/fastrepl/fastrepl/warnings.py:24: CompletionTruncatedWarning: A | 
https://docs.fastrepl.com/miscellaneous/warnings_and_errors#completiontruncated

Dataset({
    features: ['input', 'prediction'],
    num_rows: 120
})

One interesting point to note is that, due to the `position_debias_strategy="consensus"`, if the order of the labels affects the result, `fastrepl` will return `None`. We are be returning more meaningful value in the later version of `fastrepl`.

In [7]:
first_result = [row[0] for row in result["prediction"]]
second_result = [row[1] for row in result["prediction"]]

print(first_result.count(None))
print(second_result.count(None))

21
37


Now we got some numbers.

In [8]:
def metric(result):
    f = result.count("FUN")
    nf = result.count("NOT_FUN")
    return f / (f + nf)


print(metric(first_result))
print(metric(second_result))

0.46464646464646464
0.43373493975903615
